In [11]:
#%autoreload
from keras.preprocessing import image
from keras import applications
from keras.models import Sequential
from keras.applications import vgg16
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.layers import Conv2D, Conv3D,Input, ZeroPadding3D, Reshape
from keras.layers.convolutional import Convolution2D, Convolution3D, MaxPooling2D, ZeroPadding2D,ZeroPadding3D 
from keras.layers.core import Reshape
import os
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.callbacks import CSVLogger
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.optimizers import SGD
import numpy as np
import keras
import keras.backend as K
#import imp
#imp.reload(parse_data)

In [12]:
def mean_pred(y_true, y_pred):
    return keras.backend.l2_normalize(y_pred-y_true, axis=None)
   # return np.linalg.norm(y_pred-y_true)/np.linalg.norm(y_true)

In [13]:
model_vgg16 = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(299,299,3))
input_x = Input((299,299,4))
x = Conv2D(3,(3,3), activation='relu', data_format="channels_last")(input_x)
x = ZeroPadding2D((1,1))(x)
x = Reshape((299,299,3))(x)
x = model_vgg16(x)

flatten = Flatten()
new_layer2 = Dense(1024, activation='relu', name='my_dense_2')
out2 = new_layer2(flatten(x))
x = Dense(512, activation='relu')(out2)
preds = Dense(3, activation = 'relu')(x)

model_start = Model(input_x, preds)
model_start.compile(loss='mean_squared_error', optimizer=SGD(lr=1e-4, momentum=0.9),
            metrics=[mean_pred])
model_start.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 299, 299, 4)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 297, 297, 3)       111       
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 299, 299, 3)       0         
_________________________________________________________________
reshape_3 (Reshape)          (None, 299, 299, 3)       0         
_________________________________________________________________
vgg16 (Model)                (None, 9, 9, 512)         14714688  
_________________________________________________________________
flatten_3 (Flatten)          (None, 41472)             0         
_________________________________________________________________
my_dense_2 (Dense)           (None, 1024)              42468352  
__________

In [15]:
model_start.metrics_names

['loss', 'mean_pred']

In [ ]:
object_path = "/home/zhouzixuan/proj/data/train/35/"
dev_path = "/home/zhouzixuan/proj/data/dev/35/"
filename ="vgg"
rescale=1. / 255
img_width, img_height = 299, 299
# d_batch = np.load(object_path+str(19)+"_d.npy")
# x_batch = np.load(object_path+str(19)+"_x.npy")
# d_round = np.floor(d_batch/51)
# d_round = d_round.astype(np.int)
# x_batch = np.load(object_path+str(19)+"_y.npy")
# print x_batch.shape
# X = (np.concatenate((x_batch,np.expand_dims(d_batch, axis=3)), axis=3))
# print X.shape

In [ ]:
dev_num = 7
X_dev = np.empty([0,299,299,4])
y_dev = np.empty([0,3])
for k in range(dev_num):
    x_dev = np.load(dev_path+str(60+k)+"_x.npy") * rescale
    d_dev = np.load(dev_path+str(60+k)+"_d.npy")
    tmp1 = (np.concatenate((x_dev,np.expand_dims(d_dev, axis=3)), axis=3)) 
    X_dev = np.vstack([X_dev, tmp1])
    tmp2 = np.load(dev_path+str(60+k)+"_y.npy")
    y_dev = np.vstack([y_dev, tmp2])
print y_dev.shape
print X_dev.shape

In [9]:
def train(model):

    fh = open(filename + 'report.txt','w')
    csv_logger = CSVLogger('logvgg.csv', append=True, separator=';')      
    batch = 32
    epochs = 20
    batch_num = 60
    rescale=1. / 255      
    for e in range(epochs):
        print('Epoch', e)
        batches = 0
        for b in range(batch_num):
            x_batch = np.load(object_path+str(b)+"_x.npy") * rescale
            d_batch = np.load(object_path+str(b)+"_d.npy")
            X = (np.concatenate((x_batch,np.expand_dims(d_batch, axis=3)), axis=3))           
            y_batch = np.load(object_path+str(b)+"_y.npy")
            model.fit(X, y_batch,validation_data=(X_dev, y_dev))

In [10]:
train(model_start)

('Epoch', 0)
Epoch 1/1
32/32 [==============================] - 80s 2s/step - loss: 2.2666 - mean_pred: -1.3020
Epoch 1/1
32/32 [==============================] - 72s 2s/step - loss: 1.8181 - mean_pred: 7.8463
Epoch 1/1
32/32 [==============================] - 72s 2s/step - loss: 1.3569 - mean_pred: -0.8650
Epoch 1/1
32/32 [==============================] - 72s 2s/step - loss: 0.8445 - mean_pred: -1.0975
Epoch 1/1
32/32 [==============================] - 72s 2s/step - loss: 0.7041 - mean_pred: -0.9521
Epoch 1/1
32/32 [==============================] - 72s 2s/step - loss: 0.8263 - mean_pred: -0.7008
Epoch 1/1
32/32 [==============================] - 72s 2s/step - loss: 0.9460 - mean_pred: -0.5474
Epoch 1/1
32/32 [==============================] - 72s 2s/step - loss: 0.8218 - mean_pred: -0.6775
Epoch 1/1
32/32 [==============================] - 72s 2s/step - loss: 0.6031 - mean_pred: -0.9409
Epoch 1/1
32/32 [==============================] - 72s 2s/step - loss: 0.4318 - mean_pred: -1.243

KeyboardInterrupt: 